<a href="https://colab.research.google.com/github/kirkwilson/annotate_images/blob/main/annotate_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**upload files**
*   needs to be png for jpeg
*   can be a zip of images

**write text**
*   variable for text to write
*   standard position on image (bottom left corner based on percentage)

**export files**
*   export image
*   zip if multiple files

In [2]:
TEXT = 'Lorem ipsum...' #@param {type:"string"}
BUFFER_PERCENT = 0.05 #@param {type:"number"}
TEXT_HEIGHT_PERCENT = 0.05 #@param {type:"number"}

In [3]:
import os.path
import zipfile
from google.colab import files
from PIL import Image, ImageFont, ImageDraw

In [4]:
DIRECTORY = '/content/decompressed_files/'
FONT_LOCATION = 'LiberationSerif-Regular.ttf'

def upload_files():
  dict_files = files.upload()
  return dict_files

def decompress_file(file_name):
  with zipfile.ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall(DIRECTORY)

def decompress_files(files):
  for file_name in files.keys():
    decompress_file(file_name)

def upload_and_decompress():
  files = upload_files()
  decompress_files(files)

def text_bbox(image, buffer, text_height):
  right = image.width - (image.width * (buffer + buffer))
  bottom = image.height * text_height
  return (0, 0, right, bottom)
  
def text_anchor(image, buffer, text_height):
  x = image.width*buffer
  y = image.height-(image.height*(buffer+buffer+text_height))
  return (x, y)

def stroke_size(font):
  pixel_height = bbox_dimensions(font)[1]
  stroke_width = int(pixel_height*0.1)
  if stroke_width == 0:
    stroke_width = 1
  return stroke_width

def create_font(font_location, font_size=12):
  font = ImageFont.truetype(font_location, font_size)
  return font

def font_text_bbox(font, text):
  return font.getbbox(text)

def bbox_dimensions(bbox):
  width = bbox[2] - bbox[0]
  height = bbox[3] - bbox[1]
  return (width, height)

def can_a_fit_in_b(a, b):
  a = bbox_dimensions(a)
  b = bbox_dimensions(b)
  return a<=b

def annotated_file_name(file_name):
  name, extension = file_name.split('.')
  annotated = name + '_annotated.' + extension
  return annotated

# def walk_directory():
#   for root, dirs, files in os.walk(DIRECTORY):
#     for file in files:

In [5]:
file_name = 'example-img.png'

img = Image.open(file_name)

text_area = text_bbox(img, BUFFER_PERCENT, TEXT_HEIGHT_PERCENT)
anchor = text_anchor(img, BUFFER_PERCENT, TEXT_HEIGHT_PERCENT)

draw = ImageDraw.Draw(img)

font = create_font(FONT_LOCATION, 35)
font_area = font_text_bbox(font, TEXT)

if can_a_fit_in_b(font_area, text_area):
  stroke_width = stroke_size(font_area)
  draw.text(anchor, TEXT, 
            font=font, stroke_width=stroke_width, stroke_fill='black')
  img.save(annotated_file_name(file_name))

# todo:
# write function to check pixles between two sets of dimensions
# sort to to find the largest font size that fits in the text box

example-img_annotated.png
